In [ ]:
import tensorflow as tf

# Set memory growth configuration at the beginning of your script or notebook
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Now you can proceed with the rest of your code
# ...

# Model definition and training
# ...


In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU available:", gpus)
else:
    print("No GPU available.")


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 2: Data Preparation
# Unzip the dataset (adjust the file path accordingly)
!unzip -q '/content/drive/MyDrive/Data.zip' -d '/content/dataset'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define class names and the number of images per class
class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

class_image_counts = [1257, 3323, 2079, 1677, 7970, 3140, 2055, 1847, 1702, 2103]

# Organize the data into training and testing sets
base_dir = '/content/dataset/IMG_CLASSES'  # Update to your dataset directory

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Move images into train and test directories
for class_name, image_count in zip(class_names, class_image_counts):
    class_dir = os.path.join(base_dir, class_name)
    images = os.listdir(class_dir)

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Create class-specific directories in the train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(test_class_dir):
        os.makedirs(test_class_dir)

    # Move images to their respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_class_dir, image)
        shutil.move(src, dst)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(test_class_dir, image)
        shutil.move(src, dst)


In [ ]:
# Count and print the number of images in train and test directories
for class_name in class_names:
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    # Count images in the train directory
    train_files = gb.glob(pathname=os.path.join(train_class_dir, '*.jpg'))
    print(f'For training data, found {len(train_files)} images in class {class_name}')

    # Count images in the test directory
    test_files = gb.glob(pathname=os.path.join(test_class_dir, '*.jpg'))
    print(f'For testing data, found {len(test_files)} images in class {class_name}')


For training data, found 1005 images in class Atopic Dermatitis
For testing data, found 252 images in class Atopic Dermatitis
For training data, found 2658 images in class Basal Cell Carcinoma (BCC)
For testing data, found 665 images in class Basal Cell Carcinoma (BCC)
For training data, found 1663 images in class Benign Keratosis-like Lesions (BKL)
For testing data, found 416 images in class Benign Keratosis-like Lesions (BKL)
For training data, found 1341 images in class Eczema
For testing data, found 336 images in class Eczema
For training data, found 6376 images in class Melanocytic Nevi (NV)
For testing data, found 1594 images in class Melanocytic Nevi (NV)
For training data, found 2512 images in class Melanoma
For testing data, found 628 images in class Melanoma
For training data, found 1644 images in class Psoriasis pictures Lichen Planus and related diseases
For testing data, found 411 images in class Psoriasis pictures Lichen Planus and related diseases
For training data, foun

In [ ]:
train_sizes = []  # List to store image sizes in the training dataset

# Iterate through each class in the training directory
for class_name in os.listdir(train_dir):
    train_class_dir = os.path.join(train_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(train_class_dir, '*.jpg')):
        image = plt.imread(file)
        train_sizes.append(image.shape)

# Create a Pandas Series to count and display the unique image sizes
train_size_counts = pd.Series(train_sizes).value_counts()

# Print the counts of unique image sizes in the training dataset
print("Image sizes in the training dataset:")
print(train_size_counts)


Image sizes in the training dataset:
(450, 600, 3)      5976
(1024, 1024, 3)    3143
(512, 512, 3)      2512
(472, 720, 3)      1902
(222, 294, 3)      1655
                   ... 
(720, 692, 3)         1
(720, 566, 3)         1
(720, 620, 3)         1
(536, 720, 3)         1
(720, 569, 3)         1
Length: 361, dtype: int64


In [ ]:
test_sizes = []  # List to store image sizes in the training dataset

# Iterate through each class in the training directory
for class_name in os.listdir(test_dir):
    test_class_dir = os.path.join(test_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(test_class_dir, '*.jpg')):
        image = plt.imread(file)
        test_sizes.append(image.shape)

# Create a Pandas Series to count and display the unique image sizes
test_size_counts = pd.Series(test_sizes).value_counts()

# Print the counts of unique image sizes in the training dataset
print("Image sizes in the testing dataset:")
print(test_size_counts)


Image sizes in the testing dataset:
(450, 600, 3)      1485
(1024, 1024, 3)     804
(512, 512, 3)       628
(472, 720, 3)       438
(222, 294, 3)       376
                   ... 
(720, 540, 3)         1
(720, 468, 3)         1
(590, 720, 3)         1
(625, 720, 3)         1
(512, 720, 3)         1
Length: 180, dtype: int64


In [ ]:
s=400

In [ ]:
# Mapping between numeric labels and class names
code = {'Atopic Dermatitis': 0, 'Basal Cell Carcinoma (BCC)': 1, 'Benign Keratosis-like Lesions (BKL)': 2,
        'Eczema': 3, 'Melanocytic Nevi (NV)': 4, 'Melanoma': 5,
        'Psoriasis pictures Lichen Planus and related diseases': 6,
        'Seborrheic Keratoses and other Benign Tumors': 7,
        'Tinea Ringworm Candidiasis and other Fungal Infections': 8,
        'Warts Molluscum and other Viral Infections': 9}

# Function to get the class name from its numeric label
def getcode(n):
    for class_name, label in code.items():
        if n == label:
            return class_name

In [ ]:
X_train = []
y_train = []

# Iterate through each class in the training directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(class_dir, '*.jpg')):
        image = cv2.imread(file)
        image_array = cv2.resize(image, (s, s))

        X_train.append(image_array)
        y_train.append(code[class_name])

# Convert the lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Print the shape of the training data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (21719, 400, 400, 3)
y_train shape: (21719,)


In [ ]:
print(f"We have {len(X_train)} items in X_train")

We have 21719 items in X_train


In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),36))) :
    plt.subplot(6,6,n+1)
    plt.imshow(X_train[i])
    plt.axis('off')
    plt.title(getcode(y_train[i]))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
X_test = []
y_test = []

# Iterate through each class in the test directory
for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)

    # Iterate through each image in the class
    for file in gb.glob(pathname=os.path.join(class_dir, '*.jpg')):
        image = cv2.imread(file)
        image_array = cv2.resize(image, (s, s))

        X_test.append(image_array)
        y_test.append(code[class_name])

# Convert the lists to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print the shape of the test data
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print(f"We have {len(X_test)} items in X_test")


X_test shape: (5434, 400, 400, 3)
y_test shape: (5434,)
We have 5434 items in X_test


In [ ]:
plt.figure(figsize=(20, 20))
for n, i in enumerate(list(np.random.randint(0, len(X_test), 36))):
    plt.subplot(6, 6, n+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(getcode(y_test[i]))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
#X_pred_array = np.array(X_pred)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
#print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

X_train shape  is (21719, 400, 400, 3)
X_test shape  is (5434, 400, 400, 3)
y_train shape  is (21719,)
y_test shape  is (5434,)


In [ ]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)),
        keras.layers.MaxPool2D(3,3),
        keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(3,3),
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(3,3),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(3,3),
        keras.layers.Flatten() ,
        keras.layers.Dense(120,activation='relu') ,
        keras.layers.Dropout(rate=0.5) ,
        keras.layers.Dense(10,activation='softmax') ,
        ])

In [ ]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print('Model Details are : ')
print(KerasModel.summary())

Model Details are : 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 498, 498, 512)     14336     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 166, 166, 512)     0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 164, 164, 256)     1179904   
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 54, 54, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 128)       295040    
                                                                 
 max_pooling2d_4 (MaxPoolin  (Non

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


In [ ]:
def data_generator(X, y, batch_size):
    while True:
        for i in range(0, len(X), batch_size):
            yield X[i:i+batch_size], y[i:i+batch_size]

batch_size = 32
epochs = 50

train_generator = data_generator(X_train, y_train, batch_size)

ThisModel = KerasModel.fit(train_generator, epochs=epochs, steps_per_epoch=len(X_train)//batch_size, verbose=1)


In [ ]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=10)


In [ ]:
import tensorflow as tf

with tf.device('/device:GPU:0'):
    X_train_gpu = tf.convert_to_tensor(X_train)
    y_train_gpu = tf.convert_to_tensor(y_train)


In [ ]:

batch_size = 32
epochs = 50
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

InternalError: ignored

In [ ]:
batch_size = 16  # or any smaller value
epochs = 50

train_generator = data_generator(X_train, y_train, batch_size)

ThisModel = KerasModel.fit(train_generator, epochs=epochs, steps_per_epoch=len(X_train)//batch_size, verbose=1)


In [ ]:
epochs = 50
ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=32,verbose=1)

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam



In [ ]:
jkhkjbkjb


model = Sequential()

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', input_shape=(s,s, 3), padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 50
model = Sequential()
model.add(Conv2D(256, kernel_size = (3,3), input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(Conv2D(16, kernel_size = (3,3), activation = 'relu'))
model.add(Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(256,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(128,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(64,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(Dense(32,activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(NUM_CLASSES,activation='softmax'))

In [ ]:
#Step 5: Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for the test set
train_generator = datagen.flow_from_directory(train_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')

Found 21719 images belonging to 10 classes.
Found 5434 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.optimizers import Adam


custom_optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 50

model = Sequential()

model.add(Conv2D(400, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(350, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(250, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(200, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(100, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))


In [ ]:
IMAGE_WIDTH = 100
IMAGE_HEIGHT = 100
NUM_CLASSES = 10
BATCH_SIZE = 128
EPOCHS = 64
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3), kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3), kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))  # Adjust the dropout rate

model.add(Flatten())

model.add(Dense(512, activation='relu', kernel_initializer='glorot_normal'))  # Increase the number of units
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='glorot_normal'))

# model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3), kernel_initializer='glorot_normal'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Dropout(0.5))
# model.add(Flatten())

# model.add(Dense(512, activation='relu', kernel_initializer='glorot_normal'))
# model.add(Dropout(0.5))

# model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='glorot_normal'))


In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 10
BATCH_SIZE = 32
EPOCHS = 64

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
# Step 4: Compile the model
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:



# custom_optimizer = Adam(learning_rate=0.01)

# model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# optimizer=tf.keras.optimizers.Adam(lr=0.001)

# model.compile(loss = 'categorical_crossentropy',
#               optimizer =optimizer,
#               metrics = ['accuracy'])

In [ ]:
#Step 5: Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for the test set
train_generator = datagen.flow_from_directory(train_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')

Found 21719 images belonging to 10 classes.
Found 5434 images belonging to 10 classes.


In [ ]:
from keras import backend as K

K.clear_session()

In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

Epoch 1/64
170/170 [==============================] - 178s 965ms/step - loss: 1.7997 - accuracy: 0.3600 - val_loss: 1.4605 - val_accuracy: 0.4687
Epoch 2/64
170/170 [==============================] - 164s 964ms/step - loss: 1.4519 - accuracy: 0.4661 - val_loss: 1.2874 - val_accuracy: 0.5261
Epoch 3/64
170/170 [==============================] - 164s 962ms/step - loss: 1.3175 - accuracy: 0.5091 - val_loss: 1.2442 - val_accuracy: 0.5195
Epoch 4/64
170/170 [==============================] - 164s 962ms/step - loss: 1.2435 - accuracy: 0.5342 - val_loss: 1.1284 - val_accuracy: 0.5615
Epoch 5/64
170/170 [==============================] - 166s 974ms/step - loss: 1.1974 - accuracy: 0.5502 - val_loss: 1.1259 - val_accuracy: 0.5629
Epoch 6/64
170/170 [==============================] - 164s 967ms/step - loss: 1.1575 - accuracy: 0.5649 - val_loss: 1.1508 - val_accuracy: 0.5642
Epoch 7/64
170/170 [==============================] - 164s 967ms/step - loss: 1.1226 - accuracy: 0.5773 - val_loss: 1.0511 -

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# Step 4: Model Definition - Choose EfficientNetB0 as the pre-trained model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))

# Step 3: Freeze Pre-trained Layers
for layer in base_model.layers:
    layer.trainable = False

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
# Step 4: Build a New Model
from tensorflow.keras.layers import GlobalAveragePooling2D
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='glorot_normal'))


In [ ]:
# Step 5: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 6: Train the Model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

Epoch 1/64
170/170 [==============================] - 172s 977ms/step - loss: 2.1648 - accuracy: 0.2849 - val_loss: 2.1289 - val_accuracy: 0.2933
Epoch 2/64
170/170 [==============================] - 165s 969ms/step - loss: 2.1466 - accuracy: 0.2936 - val_loss: 2.1304 - val_accuracy: 0.2933
Epoch 3/64
 59/170 [=========>....................] - ETA: 1:31 - loss: 2.1429 - accuracy: 0.2924

In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

In [ ]:
# Step 3: Define model architecture
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 64
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
# Step 4: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 5: Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for the test set

train_generator = datagen.flow_from_directory(train_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')

Found 21719 images belonging to 10 classes.
Found 5434 images belonging to 10 classes.


In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

Epoch 1/50
340/340 [==============================] - 187s 539ms/step - loss: 1.6519 - accuracy: 0.4044 - val_loss: 1.3966 - val_accuracy: 0.4704
Epoch 2/50
340/340 [==============================] - 183s 539ms/step - loss: 1.2990 - accuracy: 0.5139 - val_loss: 1.1989 - val_accuracy: 0.5534
Epoch 3/50
340/340 [==============================] - 184s 541ms/step - loss: 1.1823 - accuracy: 0.5526 - val_loss: 1.1104 - val_accuracy: 0.5666
Epoch 4/50
340/340 [==============================] - 183s 538ms/step - loss: 1.1138 - accuracy: 0.5761 - val_loss: 1.0599 - val_accuracy: 0.5902
Epoch 5/50
340/340 [==============================] - 183s 540ms/step - loss: 1.0671 - accuracy: 0.5921 - val_loss: 1.0311 - val_accuracy: 0.6102
Epoch 6/50
340/340 [==============================] - 183s 540ms/step - loss: 1.0248 - accuracy: 0.6136 - val_loss: 1.0194 - val_accuracy: 0.6159
Epoch 7/50
340/340 [==============================] - 184s 540ms/step - loss: 0.9891 - accuracy: 0.6225 - val_loss: 0.9787 -

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print("Test accuracy:", test_acc)

85/85 [==============================] - 26s 303ms/step - loss: 0.8967 - accuracy: 0.7094
Test accuracy: 0.7094221711158752


In [ ]:
import matplotlib.pyplot as plt

# Manually enter the accuracy and loss values for each epoch
train_accuracy = [0.4044, 0.5139, 0.5526, 0.5761, 0.5921, 0.6136, 0.6225, 0.6333, 0.6381, 0.6496, 0.6601, 0.6661, 0.6734, 0.6772, 0.6847, 0.6874, 0.6978, 0.7024, 0.7054, 0.7125, 0.7109, 0.7222, 0.7248, 0.7288, 0.7312, 0.7363, 0.7410, 0.7487, 0.7506, 0.7555, 0.7569, 0.7648, 0.7673, 0.7740, 0.7807, 0.7840, 0.7839, 0.7855, 0.7930, 0.7972, 0.7986, 0.8023, 0.8086, 0.8121, 0.8156, 0.8179, 0.8229, 0.8244, 0.8299, 0.8337]
val_accuracy = [0.4704, 0.5534, 0.5666, 0.5902, 0.6102, 0.6159, 0.6308, 0.6428, 0.6318, 0.6647, 0.6623, 0.6608, 0.6610, 0.6730, 0.6752, 0.6781, 0.6886, 0.6886, 0.6807, 0.6989, 0.6940, 0.6918, 0.7006, 0.7015, 0.7094, 0.7175, 0.7046, 0.7190, 0.7173, 0.7295, 0.7258, 0.7247, 0.7333, 0.7240, 0.7278, 0.7245, 0.7308, 0.7376, 0.7344, 0.7420, 0.7365, 0.7459, 0.7433, 0.7499, 0.7437, 0.7448, 0.7414, 0.7398, 0.7352, 0.7560]

train_loss = [1.6519, 1.2990, 1.1823, 1.1138, 1.0671, 1.0248, 0.9891, 0.9684, 0.9478, 0.9195, 0.8983, 0.8829, 0.8691, 0.8569, 0.8313, 0.8180, 0.8043, 0.7895, 0.7792, 0.7577, 0.7603, 0.7399, 0.7281, 0.7203, 0.7080, 0.6949, 0.6828, 0.6751, 0.6623, 0.6496, 0.6412, 0.6242, 0.6146, 0.6079, 0.5917, 0.5850, 0.5762, 0.5726, 0.5503, 0.5423, 0.5401, 0.5326, 0.5186, 0.5089, 0.5015, 0.4866, 0.4760, 0.4650, 0.4611, 0.4477]
val_loss = [1.3966, 1.1989, 1.1104, 1.0599, 1.0311, 1.0194, 0.9787, 0.9471, 0.9575, 0.8965, 0.8968, 0.9046, 0.8935, 0.8854, 0.8530, 0.8479, 0.8344, 0.8257, 0.8519, 0.8106, 0.7977, 0.8168, 0.7997, 0.7906, 0.7812, 0.7626, 0.7841, 0.7523, 0.7527, 0.7378, 0.7388, 0.7433, 0.7293, 0.7442, 0.7376, 0.7392, 0.7241, 0.7152, 0.7296, 0.7139, 0.7095, 0.7082, 0.7237, 0.6948, 0


In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

Epoch 1/64
340/340 [==============================] - 157s 454ms/step - loss: 1.6495 - accuracy: 0.4070 - val_loss: 1.2736 - val_accuracy: 0.5156
Epoch 2/64
340/340 [==============================] - 155s 458ms/step - loss: 1.2807 - accuracy: 0.5171 - val_loss: 1.1668 - val_accuracy: 0.5497
Epoch 3/64
340/340 [==============================] - 155s 455ms/step - loss: 1.1892 - accuracy: 0.5465 - val_loss: 1.1694 - val_accuracy: 0.5399
Epoch 4/64
340/340 [==============================] - 155s 455ms/step - loss: 1.1220 - accuracy: 0.5770 - val_loss: 1.1260 - val_accuracy: 0.5703
Epoch 5/64
340/340 [==============================] - 155s 455ms/step - loss: 1.0804 - accuracy: 0.5874 - val_loss: 1.0424 - val_accuracy: 0.5985
Epoch 6/64
340/340 [==============================] - 155s 456ms/step - loss: 1.0445 - accuracy: 0.5992 - val_loss: 0.9721 - val_accuracy: 0.6198
Epoch 7/64
340/340 [==============================] - 154s 453ms/step - loss: 1.0221 - accuracy: 0.6090 - val_loss: 1.1000 -

In [ ]:
# Step 6: Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=EPOCHS, validation_data=test_generator, validation_steps=len(test_generator))

Epoch 1/64
340/340 [==============================] - 230s 673ms/step - loss: 1.5210 - accuracy: 0.4458 - val_loss: 1.2763 - val_accuracy: 0.5180
Epoch 2/64
340/340 [==============================] - 227s 667ms/step - loss: 1.2919 - accuracy: 0.5257 - val_loss: 1.2657 - val_accuracy: 0.5353
Epoch 3/64
340/340 [==============================] - 227s 667ms/step - loss: 1.1956 - accuracy: 0.5561 - val_loss: 1.1281 - val_accuracy: 0.5707
Epoch 4/64
340/340 [==============================] - 226s 666ms/step - loss: 1.1236 - accuracy: 0.5871 - val_loss: 1.0551 - val_accuracy: 0.6031
Epoch 5/64
340/340 [==============================] - 227s 668ms/step - loss: 1.0815 - accuracy: 0.6003 - val_loss: 1.0728 - val_accuracy: 0.5951
Epoch 6/64
340/340 [==============================] - 226s 666ms/step - loss: 1.0333 - accuracy: 0.6148 - val_loss: 0.9739 - val_accuracy: 0.6246
Epoch 7/64
340/340 [==============================] - 226s 665ms/step - loss: 1.0132 - accuracy: 0.6212 - val_loss: 1.0005 -